In [8]:
import os
!git clone https://github.com/arapgechina24-lgtm/nctirs-platform-v2.git
%cd nctirs-platform-v2/ai_training
!pip install -r requirements.txt

Cloning into 'nctirs-platform-v2'...
remote: Enumerating objects: 8256, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 8256 (delta 52), reused 42 (delta 41), pack-reused 8191 (from 2)
Receiving objects: 100% (8256/8256), 44.31 MiB | 13.09 MiB/s, done.
Resolving deltas: 100% (1604/1604), done.
/content/nctirs-platform-v2/ai_training/nctirs-platform-v2/ai_training


# NCTIRS Platform V2 - AI Training Pipeline
This notebook automatically clones your repository, installs dependencies, and runs the entire training and export pipeline.

In [7]:
# 1. Download dataset
!python download_datasets.py

# 2. Generate synthetic data if needed
!python generate_synthetic.py


############################################################
# DATASET: CICIDS2017 (Canadian Institute for Cybersecurity)
# 8 attack types, 2.8M+ flows, CSE/UNB collaboration
############################################################

Downloading: cicids2017_MachineLearningCSV.zip
Destination: /content/nctirs-platform-v2/ai_training/data/cicids2017/cicids2017_MachineLearningCSV.zip
  File already exists: /content/nctirs-platform-v2/ai_training/data/cicids2017/cicids2017_MachineLearningCSV.zip (0.1 MB)

  Extracting cicids2017_MachineLearningCSV.zip...
  ✗ Extraction failed: File is not a zip file

############################################################
# DATASET: UNSW-NB15 (University of New South Wales)
# 10 attack categories, 2.5M+ records, ACCS research
############################################################

Downloading: UNSW_NB15_training-set.csv
Destination: /content/nctirs-platform-v2/ai_training/data/unsw_nb15/UNSW_NB15_training-set.csv

  ✗ Download failed: HTTP E

In [3]:
# 3. Train the model (Using the combined pipeline)
!python train_combined.py

  ✗ UNSW-NB15 not found at /content/nctirs-platform-v2/ai_training/data/unsw_nb15/unsw_nb15_full.csv
  ✗ CICIDS2017 not found at /content/nctirs-platform-v2/ai_training/data/cicids2017/cicids2017_full.csv

[3/3] Loading Synthetic dataset...
  ✓ Synthetic: 100000 samples (80000 normal, 20000 attack)
  Attack types: {'Botnet': 2000, 'WebAttack': 2000, 'PortScan': 2000, 'Infiltration': 2000, 'DNSTunnel': 2000, 'Heartbleed': 2000, 'BruteForce': 2000, 'DataExfil': 2000, 'C2Beacon': 2000, 'DDoS': 2000}

COMBINED DATASET: 100000 total samples
  Normal: 80000 (80.0%)
  Attack: 20000 (20.0%)
  Features: 46

Train: 69991 | Val: 14991 | Test: 14991
Model: 71,472 parameters | Device: cpu

TRAINING — 100 epochs, batch 256, LR 0.001

Epoch    1/100 | Train: 0.6824 acc=0.8991 | Val: 0.6082 acc=0.9984 | LR: 1.00e-03 | 24s
Epoch    5/100 | Train: 0.5705 acc=0.9987 | Val: 0.5559 acc=0.9993 | LR: 1.00e-03 | 119s
Epoch   10/100 | Train: 0.5493 acc=0.9993 | Val: 0.5386 acc=0.9994 | LR: 1.00e-03 | 236s
Epoc

In [12]:
# 4. Evaluate the trained models
!python evaluate.py

Loaded checkpoint from epoch 3 (val_loss: 0.2340)
Traceback (most recent call last):
  File "/content/nctirs-platform-v2/ai_training/nctirs-platform-v2/ai_training/evaluate.py", line 227, in <module>
    main()
  File "/content/nctirs-platform-v2/ai_training/nctirs-platform-v2/ai_training/evaluate.py", line 181, in main
    features, labels = loader.load_synthetic()
                       ^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/nctirs-platform-v2/ai_training/nctirs-platform-v2/ai_training/dataset_loader.py", line 145, in load_synthetic
    df = pd.read_csv(path)
         ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

In [11]:
# 5. Export weights for use in the web app
!python export_weights.py

Loaded checkpoint: epoch 3, val_loss 0.2340

Exporting Model: SENTINEL-OMEGA-ADv2 v2.0.0
Output: /content/nctirs-platform-v2/ai_training/nctirs-platform-v2/public/models/anomaly-detector-v2

Model exported to /content/nctirs-platform-v2/ai_training/nctirs-platform-v2/public/models/anomaly-detector-v2/model.json
Weights saved to /content/nctirs-platform-v2/ai_training/nctirs-platform-v2/public/models/anomaly-detector-v2/weights.bin (282.2 KB)
Normalization stats exported to /content/nctirs-platform-v2/ai_training/nctirs-platform-v2/public/models/anomaly-detector-v2/normalization_stats.json
Model metadata exported to /content/nctirs-platform-v2/ai_training/nctirs-platform-v2/public/models/anomaly-detector-v2/model_metadata.json
Evaluation report copied to /content/nctirs-platform-v2/ai_training/nctirs-platform-v2/public/models/anomaly-detector-v2/evaluation_report.json

Export complete! Files in /content/nctirs-platform-v2/ai_training/nctirs-platform-v2/public/models/anomaly-detector-v2:

In [9]:
# 6. Download the generated weights to your local machine
from google.colab import files
import os

print("Zipping checkpoints...")
!zip -r model_checkpoints.zip checkpoints/

if os.path.exists('model_checkpoints.zip'):
    print("Downloading checkpoints...")
    files.download('model_checkpoints.zip')
else:
    print("Could not find model_checkpoints.zip")

Zipping checkpoints...
  adding: checkpoints/ (stored 0%)
  adding: checkpoints/best_model.pt (deflated 10%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>